<a href="https://colab.research.google.com/github/ja8an/big-data/blob/main/TDE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Run below commands in google colab
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark3.0.0
!wget -q https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
# unzip it
!tar xf spark-3.3.0-bin-hadoop3.tgz
# install findspark 
!pip install -q findspark
!pip install -q pyspark

!echo "finished :)"

finished :)


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"

!echo "env set :)"

env set :)


In [ ]:
import findspark
findspark.init()
from pyspark import SparkConf, SparkContext, SparkFiles

conf = SparkConf().setAppName("teste").setMaster("local[*]")

sc = SparkContext.getOrCreate(conf = conf)

"spark setup"

'spark setup'

In [ ]:
dataset = sc.textFile("/content/transactions.csv")
dataset = dataset.flatMap(lambda line : line.split('\n'))
dataset = dataset.filter(lambda line : line.strip() != 'country_or_area;year;comm_code;commodity;flow;trade_usd;weight_kg;quantity_name;quantity;category')
dataset = dataset.map(lambda cols : cols.split(';'))

In [ ]:
def sum(a, b):
  if type(a) is tuple and type(b) is tuple:
    return (a[0] + b[0], a[1] + b[1])
  return a + b

(Easy) The number of transactions involving Brazil;

In [ ]:
dataset.map(lambda cols : (cols[0], 1)).filter(lambda a : a[0] == 'Brazil').reduceByKey(sum).first()

('Brazil', 26918)

(Easy) The number of transactions per year;

In [ ]:
dataset.map(lambda cols : (cols[1], 1)).reduceByKey(sum).take(5)

[('2016', 40225),
 ('2011', 51746),
 ('2003', 49388),
 ('1993', 20426),
 ('2007', 51806)]

(Easy) The number of transactions per flow type and year;

In [ ]:
dataset.map(lambda cols: ((cols[4], cols[1]), 1)).reduceByKey(sum).take(5)

[(('Import', '2013'), 30070),
 (('Import', '2010'), 30762),
 (('Import', '2003'), 29341),
 (('Export', '2015'), 17370),
 (('Export', '2014'), 17609)]

(Easy) The average of commodity values per year;

# Aqui agrupa por commodity e ano? não entendir

In [ ]:
_map = dataset.map(lambda cols: (cols[1], (float(cols[5]), 1)))
_sum = _map.reduceByKey(sum)
_mean = _sum.mapValues(lambda a: a[0] / a[1])
_mean.take(5)

[('2016', 29658304.060758233),
 ('2011', 33275375.636087812),
 ('2003', 13082967.865939094),
 ('1993', 10283984.1527465),
 ('2007', 23660350.780952014)]

(Easy) The average price of commodities per unit type, year, and category in the export flow in Brazil;

In [ ]:
_filter = dataset.filter(lambda cols: cols[0] == 'Brazil' and cols[4] == 'Export')
_map = _filter.map(lambda cols: ((cols[7], cols[1], cols[9]), (float(cols[5]), 1)))
_sum = _map.reduceByKey(sum)
_mean = _sum.mapValues(lambda a: a[0] / a[1])
_mean.take(5)

[(('Weight in kilograms', '1990', '87_vehicles_other_than_railway_tramway'),
  56543736.2),
 (('Weight in kilograms',
   '2010',
   '23_residues_wastes_of_food_industry_animal_fodder'),
  76.0),
 (('Weight in kilograms', '1993', '65_headgear_and_parts_thereof'), 340288.5),
 (('Weight in kilograms', '2009', '72_iron_and_steel'), 12820701.181818182),
 (('Weight in kilograms', '1993', '83_miscellaneous_articles_of_base_metal'),
  2636002.2)]

(Medium) The maximum, minimum, and mean transaction price per unit type and year;

In [ ]:
_map = dataset.map(lambda cols: ((cols[7], cols[1]), (float(cols[5]), 1, float(cols[5]), float(cols[5]))))
_sum = _map.reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1], max(a[2], b[2]), min(a[3], b[3])))
_mean = _sum.mapValues(lambda a: {
    'mean': a[0] / a[1],
    'max': a[2],
    'min': a[3]
})
_mean.take(5)

[(('Weight in kilograms', '1996'),
  {'mean': 10913516.227066854, 'max': 9884802991.0, 'min': 1.0}),
 (('Weight in kilograms', '1997'),
  {'mean': 8007879.837024995, 'max': 2646628564.0, 'min': 1.0}),
 (('Weight in kilograms', '2001'),
  {'mean': 9043813.308666354, 'max': 10391536933.0, 'min': 1.0}),
 (('Weight in kilograms', '2005'),
  {'mean': 18259614.2783081, 'max': 45281302495.0, 'min': 1.0}),
 (('Weight in kilograms', '2006'),
  {'mean': 19949033.104918696, 'max': 66411903451.0, 'min': 1.0})]

(Hard) The most commercialized commodity (summing the quantities) in 2016, per flow type.

In [ ]:
_filter = dataset.filter(lambda cols: cols[1] == '2016') # Only 2016
_map = _filter.map(lambda a: ((a[4], a[3]), float(a[8])))
_sum = _map.reduceByKey(sum)
_map = _sum.map(lambda a: (a[0][0], (a[0][1], a[1])))
_max = _map.reduceByKey(lambda a, b: a if a[1] > b[1] else b)
_map = _max.mapValues(lambda a: a[0])
_map.collect()

[('Export', 'Iron ore, concentrate, not iron pyrites,unagglomerate'),
 ('Import', 'Petroleum oils, oils from bituminous minerals, crude'),
 ('Re-Export', 'Safety razor blades, including blanks in strips'),
 ('Re-Import', 'Chem wood pulp, soda/sulphate, non-conifer, bleached')]